# ANALYSIS.
## Producer price DB.

In [2]:
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

import importlib
import src.analysis as an
import sys 
an = sys.path.append("\src\\analysis.py")

ModuleNotFoundError: No module named 'src'

IMPORT roducer price data set.

In [34]:
price= pd.read_csv("D:\ironhack\proyectos\_proy4\csv\\food_production\producer_price.csv")
price.sample()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Months Code,Months,Unit,Value
10879,PP,Producer Prices,504,Morocco,5532,Producer Price (USD/tonne),1354.0,Strawberries,2014,2014,7021,Annual value,USD,1025.4


Clean Data Base.

In [7]:
# Drop columns.
price.drop(columns=["Domain", "Domain Code", "Element Code", "Element", "Year Code", "Months", "Months Code"],inplace=True)

In [8]:
# Change names.
price.rename(columns={"Area Code (M49)":"Area Code", "Item Code (CPC)": "Item Code"}, inplace= True)

In [9]:
def iterate_column_years (df, year):
    # iterate 2010
    x = {index:[row[0],row[1],row[2],row[3],row[6]] for index, row in df.iterrows() if row[4] == year}
    df = pd.DataFrame.from_dict(x, orient="index")
    df.rename(columns= {0: "Area Code", 1:"Area", 2:"Item Code", 3:"Item", 4:year}, inplace = True)
    df["Item"] = df["Item"].astype(object)
    df["Area"] = df["Area"].astype(object)
    df[year] = df[year].astype(object)
    
    return df.drop_duplicates(subset=['Area Code', 'Item Code'], keep='last')

Iteration throw each year and create column with year.

In [10]:
# iterate 2010
df2010 = iterate_column_years (price, 2010)

In [11]:
# iterate 2011
df2011 = iterate_column_years (price, 2011)

In [12]:
# iterate 2012
df2012 = iterate_column_years (price, 2012)

In [13]:
# iterate 2013
df2013 = iterate_column_years (price, 2013)

In [14]:
# iterate 2014
df2014 = iterate_column_years (price, 2014)

In [15]:
# iterate 2015
df2015 = iterate_column_years (price, 2015)

In [16]:
# iterate 2016
df2016 = iterate_column_years (price, 2016)

In [17]:
# iterate 2017
df2017 = iterate_column_years (price, 2017)

In [18]:
# iterate 2018
df2018 = iterate_column_years (price, 2018)

In [19]:
# iterate 2019
df2019 = iterate_column_years (price, 2019)

In [20]:
# iterate 2020
df2020 = iterate_column_years (price, 2020)

Drop columns of production and duplicates, and change types.

In [21]:
def clean2_df (df):
    df.drop(columns=["Year", "Value", "Unit"],inplace=True)
    df["Item Code"] = df["Item Code"].astype(object)
    df["Area Code"] = df["Area Code"].astype(object)
    # drop duplicates.
    df.drop_duplicates(inplace=True)
    return df

In [22]:
clean2_df (price)

,Area Code,Area,Item Code,Item
0,8,Albania,1341.0,Apples
11,8,Albania,115.0,Barley
22,8,Albania,231.0,"Hen eggs in shell, fresh"
33,8,Albania,112.0,Maize (corn)
44,8,Albania,21111.01,"Meat of cattle with the bone, fresh or chilled"
...,...,...,...,...
18189,716,Zimbabwe,1510.0,Potatoes
18192,716,Zimbabwe,2211.0,Raw milk of cattle
18193,716,Zimbabwe,1802.0,Sugar cane
18196,716,Zimbabwe,1234.0,Tomatoes


Merge all data frames to 1.

In [23]:
def merging_price (df, df2010, df2011, df2012, df2013, df2014, df2015, df2016, df2017, df2018, df2019, df2020):
    z = pd.merge(df, df2010, how="left").merge(df2011, how="left").merge(df2012, how="left")
    z = pd.merge(z, df2013, how="left").merge(df2014, how="left").merge(df2015, how="left")
    z = pd.merge(z, df2016, how="left").merge(df2017, how="left").merge(df2018, how="left")        
    z = pd.merge(z, df2019, how="left").merge(df2020, how="left") 
    return z

In [24]:
df = merging_price (price, df2010, df2011, df2012, df2013, df2014, df2015, df2016, df2017, df2018, df2019, df2020)

In [31]:
def replace_items (df):
    import re
    # Clean with regex column "Items" to match other Data Frames.
    df["Item"] = df["Item"].str.replace("n\.e\.c\.", "", regex=True)
    df["Item"] = df["Item"].str.replace(".*Other berries and fruits of the genus vaccinium.*", 'Berries & Grapes', regex=True)
    df["Item"] = df["Item"].str.replace('.*Onions and shallots, green*.', 'Onions & Leeks', regex=True)
    df["Item"] = df["Item"].str.replace(".*Beer of barley, malted*.", 'Beer', regex=True)
    df["Item"] = df["Item"].str.replace(".*Barley*.", 'Beer', regex=True)
    df["Item"] = df["Item"].str.replace('.*Brazil nuts, in shell*.', "Nuts", regex=True)
    df["Item"] = df["Item"].str.replace(".*Cocoa beans*.", "Chocolate", regex=True)
    df["Item"] = df["Item"].str.replace('.*Coffee, green*.', "Coffee", regex=True)
    df["Item"] = df["Item"].str.replace('.*Groundnuts, excluding shelled*.', "Groundnuts", regex=True)
    df["Item"] = df["Item"].str.replace('.*Edible roots and tubers with high starch or inulin content, , fresh*.', 'Root Vegetables', regex=True)
    df["Item"] = df["Item"].str.replace('.*Oats*.', "Oatmeal", regex=True)
    df["Item"] = df["Item"].str.replace('.*Other citrus fruit,*.', 'Citrus Fruit', regex=True)
    df["Item"] = df["Item"].str.replace('.*Other fruits*.', 'Other Fruit', regex=True)
    df["Item"] = df["Item"].str.replace('.*Meat of pig with the bome, fresh or chilled', "Pig", regex=True)
    df["Item"] = df["Item"].str.replace('.*Meat of cattle with the bone, fresh or chilled', 'Beef', regex=True)
    df["Item"] = df["Item"].str.replace('.*Walnuts, in shell*.', 'Nuts', regex=True)
    df["Item"] = df["Item"].str.replace('.*Hazelnuts, in shell*.', 'Nuts', regex=True)
    df["Item"] = df["Item"].str.replace('.*Peas, dry*.', 'Peas', regex=True)
    df["Item"] = df["Item"].str.replace('.*Sugar beet*.', 'Beet Sugar', regex=True)
    df["Item"] = df["Item"].str.replace('.*Soya beans*.', 'Beet Sugar', regex=True)
    df["Item"] = df["Item"].str.replace('.*Other vegetables, fresh *.', 'Other Vegetables', regex=True)
    df["Item"] = df["Item"].str.replace('.*Hen eggs in shell, fresh*.', 'Eggs', regex=True)
    df["Item"] = df["Item"].str.replace('.*Gooseberries*.', 'Berries & Grapes', regex=True)
    df["Item"] = df["Item"].str.replace('.*Raw hides and skins of sheep or lambs*.', 'Lamb & Mutton', regex=True)
    df["Item"] = df["Item"].str.replace('.*Strawberries*.', 'Berries & Grapes', regex=True)
    df["Item"] = df["Item"].str.replace('.*Cranberries*.', 'Berries & Grapes', regex=True)
    df["Item"] = df["Item"].str.replace('.*Raw milk of cattle*.', 'Milk', regex=True)
    df["Item"] = df["Item"].str.replace('.*Cassava, fresh*.', 'Cassava', regex=True)
    df["Item"] = df["Item"].str.replace('.*Meat of pig with the bone, fresh or chilled*.', 'Pig Meat', regex=True)
    df["Item"] = df["Item"].str.replace('.*Chick peas, dry*.', 'Peas', regex=True)
    df["Item"] = df["Item"].str.replace('.*Sunflower-seed oil, crude*.', 'Sunflower Oil', regex=True)
    df["Item"] = df["Item"].str.replace('.*Meat of chickens, fresh or chilled*.', 'Poultry Meat', regex=True)
    df["Item"] = df["Item"].str.replace('.*Rapeseed or canola oil, crude*.', 'Rapeseed Oil', regex=True)
    df["Item"] = df["Item"].str.replace('.*Peas, green*.', 'Peas', regex=True)
    df["Item"] = df["Item"].str.replace('.*Cow peas, dry*.', 'Peas', regex=True)
    df["Item"] = df["Item"].str.replace('.*Raspberries*.', 'Berries & Grapes', regex=True)
    df["Item"] = df["Item"].str.replace('.*Blueberries*.', 'Berries & Grapes', regex=True)
    df["Item"] = df["Item"].str.replace('.*Cheese from whole cow milk*.', 'Cheese', regex=True)
    df["Item"] = df["Item"].str.title()
    df["Item"] = df["Item"].str.strip()
    return df

In [32]:
df = replace_items (df)

In [33]:
print(set(df["Item"]))

{'Chocolate', 'Beer', 'Wheat', 'Nuts', 'Oatmealealeal', 'Peas', 'Cassava', 'Beet Sugar', 'Milk', 'Potatoes', 'Rice', 'Tomatoes', 'Groundnuts', 'Poultry Meat', 'Other Fruit', 'Berries & Grapes', 'Bananas', 'Sunflower Seed', 'Palm Oil', 'Other Pulses', 'Cashew Nuts, In Shell', 'Coffee', 'Onions & Leeks', 'Apples', 'Citrus Fruit', 'Maize (Corn)', 'Eggs', 'Sugar Cane', 'Other Oil Seeds,', 'Almonds, In Shell', 'Other Vegetables', 'Root Vegetables', 'Beef', 'Pig Meat'}


Export to csv.

In [36]:
df.to_csv("D:\ironhack\proyectos\_proy4\csv\csv_from_python\producer_price.csv", index=False)